# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
llm_model = "gpt-4"

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [4]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [5]:
from langchain.indexes import VectorstoreIndexCreator

In [6]:
#pip install docarray

In [7]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

/home/mark/anaconda3/envs/OpenAI/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [9]:
response = index.query(query)

In [10]:
display(Markdown(response))



| Name | Description | Sun Protection Rating |
| --- | --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | Made of 100% polyester, UPF 50+ rated, wrinkle-resistant, front and back cape venting, two front bellows pockets, imported | SPF 50+, blocks 98% of harmful UV rays |
| Men's Plaid Tropic Shirt, Short-Sleeve | Made of 52% polyester and 48% nylon, UPF 50+ rated, SunSmart technology, wrinkle-free, front and back cape venting, two front bellows pockets, imported | SPF 50+, blocks 98% of harmful UV rays |
| Men's TropicVibe Shirt, Short-Sleeve | Made of 71% nylon and 29% polyester, UPF 50+ rated, wrinkle-resistant, front and back cape venting, two front bellows pockets, imported | SPF 50+, blocks 98% of harmful UV rays |
| Sun Shield Shirt | Made of 78% nylon and 22% Lycra Xtra Life fiber, UPF 50+ rated, moisture-wicking, fits comfortably over swimsuit, abrasion-resistant, imported | SPF

## Step By Step

In [11]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [12]:
docs = loader.load()

In [13]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [14]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [15]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [16]:
print(len(embed))

1536


In [17]:
print(embed[:5])

[-0.0219938028518557, 0.006747527976699378, -0.018252847709138532, -0.03916704653175717, -0.013997197145759583]


In [18]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [19]:
query = "Please suggest a shirt with sunblocking"

In [20]:
docs = db.similarity_search(query)

In [21]:
len(docs)

4

In [22]:
docs[0]

Document(page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255})

In [23]:
retriever = db.as_retriever()

In [24]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

/home/mark/anaconda3/envs/OpenAI/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [25]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [26]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


/home/mark/anaconda3/envs/OpenAI/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `call_as_llm` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [27]:
display(Markdown(response))

| Shirt Name | Description | Fabric Composition | Sun Protection | Additional Features |
| --- | --- | --- | --- | --- |
| Sun Shield Shirt | High-performance sun shirt that protects from harmful UV rays. Slightly fitted and falls at hip. | 78% nylon, 22% Lycra Xtra Life fiber | UPF 50+ | Wicks moisture for quick-drying comfort. Fits comfortably over swimsuit. Abrasion resistant. |
| Men's Plaid Tropic Shirt, Short-Sleeve | Ultracomfortable sun protection shirt designed for fishing and extended travel. Lightest hot-weather shirt with wrinkle-free and quick evaporation fabric. | 52% polyester, 48% nylon | UPF 50+ | Front and back cape venting, two front bellows pockets. Wrinkle-free. |
| Men's TropicVibe Shirt, Short-Sleeve | Lightweight sun-protection shirt with built-in UPF 50+. Traditional fit. | Shell: 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh | UPF 50+ | Wrinkle resistant. Front and back cape venting, two front bellows pockets. |
| Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt with superior sun protection. Traditional fit. Wrinkle-resistant. | 100% polyester | UPF 50+ | Front and back cape venting, two front bellows pockets. Wrinkle-resistant. |

All shirts provide the highest rated sun protection (UPF 50+), blocking 98% of the sun's harmful rays. They are designed for hot weather and have features like moisture-wicking, quick-drying, and wrinkle resistance. Some also have venting for cool breezes and pockets for convenience.

In [28]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [29]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [30]:
response = qa_stuff.run(query)

/home/mark/anaconda3/envs/OpenAI/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


In [31]:
display(Markdown(response))

Sure, here is a summary of the shirts with sun protection:

| Product ID | Name | Material | Sun Protection | Special Features |
| --- | --- | --- | --- | --- |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | 100% Polyester | UPF 50+ | Wrinkle-resistant, front and back cape venting, two front bellows pockets |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | 52% Polyester, 48% Nylon | UPF 50+ | Wrinkle-free, quickly evaporates perspiration, front and back cape venting, two front bellows pockets |
| 535 | Men's TropicVibe Shirt, Short-Sleeve | Shell: 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh | UPF 50+ | Wrinkle resistant, front and back cape venting, two front bellows pockets |
| 255 | Sun Shield Shirt | 78% Nylon, 22% Lycra Xtra Life fiber | UPF 50+ | Wicks moisture for quick-drying comfort, fits comfortably over your favorite swimsuit, abrasion resistant |

In [32]:
response = index.query(query, llm=llm)

In [33]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

Reminder: Download your notebook to you local computer to save your work.